<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=e135358995231eb34a116fe1b4185e62015794c281aacef4a8fd2b540c28f926
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-31 13:07:22
-------------------
qualified stocks: 87
with latest results: 26
still star stocks: 16
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  2.15 L
Current:  1.45 C
-------------------
Today PnL: -53.25 K (-0.37%)
Current PnL: -19.30 L (-12.64%)
CY Booked + Current PnL: -6.82 L (-4.47%)
-------------------
Total profit:  2.43 L
Total loss:  -21.73 L
-------------------
Total Booked + Current PnL: 20.16 L (16.05%)
Total Booked PnL: 39.46 L (31.41%)
Curr Year Booked PnL: 12.48 L (8.59%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 86.06 L (59.23%)
Deployed:  1.26 C
Current:  1.45 C
CAGR/XIRR %: 8.64%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
58,RELIANCE,1533.00,-6.91,70.0,X-LC,4.93,232643.0,22277.0,6514.0,0.19,10.59,2.80,13.68,37.0,3.42,1.63,28.86,XY25,NTT,REFINERIES
78,TTKPRESTIG,770.00,102.99,54.0,M-SC,1.45,87449.0,-13328.0,13423.0,-1.92,-13.23,15.35,0.09,245.0,-0.99,0.61,14.89,OX40N,NTT,DURABLES
79,UNIONBANK,163.00,-5.43,66.0,M-LC,10.03,171414.0,30574.0,13919.0,5.93,21.71,8.12,31.59,66.0,2.20,1.20,53.95,XY24,NTT,BANKS
41,ITC,452.00,-36.23,64.0,X-LC,1.22,205918.0,5780.0,15114.0,0.56,2.89,7.34,10.44,4.0,0.38,1.44,8.61,X40,NTT,FMCG
33,ICICIGI,2252.93,-13.14,63.0,X-MC,4.85,149480.0,13347.0,17235.0,0.29,9.80,11.53,22.47,91.0,0.77,1.04,24.06,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,UNIONBANK,163.0,-5.43,66.0,M-LC,10.03,171414.0,30574.0,13919.0,5.93,21.71,8.12,31.59,66.0,2.20,1.20,53.95,XY24,NTT,BANKS
80,UNITDSPR,1644.0,-5.09,74.0,X-LC,4.55,248798.0,14654.0,33961.0,3.72,6.26,13.65,20.77,86.0,0.43,1.74,13.61,X40N,NTT,BREWERIES
69,STARHEALTH,761.0,21.02,61.0,H-SC,13.48,269362.0,-11761.0,144620.0,2.93,-4.18,53.69,47.26,144.0,-0.08,1.88,44.32,XY24,NTT,INSURANCE
64,SFL,1287.0,38.88,64.0,M-SC,5.53,183464.0,-78754.0,148587.0,2.39,-30.03,80.99,26.63,229.0,-0.53,1.28,14.07,XY24,NTT,MISC
3,ACC,3906.0,-35.90,62.0,X-MC,2.48,190040.0,-47911.0,200568.0,2.21,-20.13,105.54,64.15,174.0,-0.24,1.33,6.60,XY24,BTT,CEMENT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BANDHANBNK,400.00,-15.36,40.0,H-SC,3.37,212176.0,-66383.0,323823.0,-7.18,-23.83,152.62,92.42,151.0,-0.20,1.48,22.62,XY24,NTT,BANKS
83,VBL,671.64,-16.58,54.0,X-LC,8.01,300498.0,-15344.0,128012.0,-2.94,-4.86,42.60,35.67,5.0,-0.12,2.10,8.32,X40N,ATH,FMCG
21,DABUR,735.00,-12.48,38.0,X-MC,3.47,237044.0,-11024.0,119423.0,-2.55,-4.44,50.38,43.70,102.0,-0.09,1.66,9.36,XY24,BTT,FMCG
61,SAIL,228.00,49.16,59.0,M-MC,10.23,236000.0,11038.0,156610.0,-2.49,4.91,66.36,74.53,192.0,0.07,1.65,38.74,XY24,BTT,STEEL
35,IEX,219.00,-33.40,46.0,H-SC,16.05,190360.0,-9594.0,106830.0,-2.28,-4.80,56.12,48.63,115.0,-0.09,1.33,6.20,XR,NTT,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
86,WIPRO,420.0,-14.7,45.0,M-LC,5.3,149891.0,-1054.0,110515.0,-0.07,-0.7,73.73,72.51,53.0,-0.01,1.05,5.28,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.0,102.99,54.0,M-SC,1.45,87449.0,-13328.0,13423.0,-1.92,-13.23,15.35,0.09,245.0,-0.99,0.61,14.89,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.42,50.0,H-SC,2.84,224730.0,-44937.0,81262.0,-0.32,-16.66,36.16,13.47,138.0,-0.55,1.57,14.27,XY24,NTT,PAINTS
17,CERA,9475.0,-21.96,54.0,H-SC,2.30,142128.0,-33775.0,75797.0,-0.07,-19.20,53.33,23.89,149.0,-0.45,0.99,22.75,OX40N,NTT,CERAMICS
73,TATAELXSI,9161.0,-22.87,51.0,H-MC,6.16,104329.0,-27506.0,69734.0,-0.88,-20.86,66.84,32.03,98.0,-0.39,0.73,16.86,OX40N,NTT,IT
67,SIS,528.0,2009.84,41.0,H-SC,3.29,85144.0,-25888.0,49494.0,-0.92,-23.32,58.13,21.26,156.0,-0.52,0.59,15.04,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,UNIONBANK,163.0,-5.43,66.0,M-LC,10.03,171414.0,30574.0,13919.0,5.93,21.71,8.12,31.59,66.0,2.2,1.2,53.95,XY24,NTT,BANKS


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAIL,228.00,49.16,59.0,M-MC,10.23,236000.0,11038.0,156610.0,-2.49,4.91,66.36,74.53,192.0,0.07,1.65,38.74,XY24,BTT,STEEL
56,RECLTD,446.00,47.87,54.0,M-LC,5.19,207158.0,3988.0,38138.0,-0.44,1.96,18.41,20.74,55.0,0.10,1.45,8.96,XY25,NTT,FINANCE
31,HINDZINC,730.22,27.24,48.0,M-LC,11.32,207756.0,2680.0,109882.0,-1.24,1.31,52.89,54.89,52.0,0.02,1.45,25.72,X5K,ATH,METALS
46,KPIGREEN,731.05,12.48,74.0,H-SC,5.26,134203.0,8906.0,50017.0,-0.35,7.11,37.27,47.03,141.0,0.18,0.94,68.03,MH,ATH,POWER
79,UNIONBANK,163.00,-5.43,66.0,M-LC,10.03,171414.0,30574.0,13919.0,5.93,21.71,8.12,31.59,66.0,2.20,1.20,53.95,XY24,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
46,KPIGREEN,731.05,12.48,74.0,H-SC,5.26,134203.0,8906.0,50017.0,-0.35,7.11,37.27,47.03,141.0,0.18,0.94,68.03,MH,ATH,POWER
85,WHIRLPOOL,2270.00,-38.01,62.0,M-SC,4.03,128450.0,13652.0,80384.0,-1.74,11.89,62.58,81.92,223.0,0.17,0.90,52.06,XR,NTT,DURABLES
36,INDIAMART,4810.62,-53.33,55.0,H-SC,6.52,130401.0,7065.0,124559.0,-0.37,5.73,95.52,106.72,119.0,0.06,0.91,30.45,AR,ATH,MISC
31,HINDZINC,730.22,27.24,48.0,M-LC,11.32,207756.0,2680.0,109882.0,-1.24,1.31,52.89,54.89,52.0,0.02,1.45,25.72,X5K,ATH,METALS
86,WIPRO,420.00,-14.70,45.0,M-LC,5.30,149891.0,-1054.0,110515.0,-0.07,-0.70,73.73,72.51,53.0,-0.01,1.05,5.28,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TATAMOTORS,1065.00,-52.51,20.0,X-LC,22.16,149719.0,-126804.0,236870.0,0.27,-45.86,158.21,39.81,54.0,-0.54,1.05,5.53,XY24,NTT,AUTO
12,BATAINDIA,2096.00,-40.58,28.0,X-SC,5.36,88785.0,-39885.0,85180.0,-0.21,-31.00,95.94,35.20,219.0,-0.47,0.62,1.91,X40,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-11.46,32.0,X-MC,2.37,87465.0,-3213.0,18123.0,-1.50,-3.54,20.72,16.44,101.0,-0.18,0.61,15.92,X40,ATH,PHARMA
44,JSWINFRA,342.00,-21.50,34.0,X-MC,5.10,197235.0,-3112.0,33609.0,-1.08,-1.55,17.04,15.23,178.0,-0.09,1.38,30.20,X40N,NTT,REALTY
23,DMART,5391.45,-17.97,35.0,X-LC,9.35,100320.0,-2266.0,29073.0,0.45,-2.21,28.98,26.13,38.0,-0.08,0.70,22.80,X40N,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,ITC,452.00,-36.23,64.0,X-LC,1.22,205918.0,5780.0,15114.0,0.56,2.89,7.34,10.44,4.0,0.38,1.44,8.61,X40,NTT,FMCG
66,SIEMENS,4671.50,-3.25,45.0,H-LC,1.22,155650.0,-30445.0,77918.0,-0.44,-16.36,50.06,25.51,15.0,-0.39,1.09,14.70,AR,ATH,ELECTRICAL
52,PGHH,17907.65,-30.55,38.0,X-MC,2.01,201135.0,315.0,67481.0,-0.03,0.16,33.55,33.76,80.0,0.00,1.41,5.36,X40,ATH,FMCG
28,HAVELLS,2069.16,-5.47,49.0,X-MC,2.88,315210.0,-12781.0,119307.0,-0.35,-3.90,37.85,32.48,92.0,-0.11,2.20,7.08,X40,ATH,ELECTRICAL
84,VOLTAS,1530.00,0.32,49.0,X-MC,4.49,210600.0,18858.0,18891.0,-1.11,9.84,8.97,19.69,99.0,1.00,1.47,17.30,XY25,NTT,AC


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BATAINDIA,2096.00,-40.58,28.0,X-SC,5.36,88785.0,-39885.0,85180.0,-0.21,-31.00,95.94,35.20,219.0,-0.47,0.62,1.91,X40,NTT,FOOTWEAR
53,QUESS,986.00,-47.11,44.0,X-SC,37.82,52768.0,-12238.0,163164.0,-2.07,-18.83,309.21,232.18,198.0,-0.08,0.37,2.93,XY24,NTT,MISC
51,PAGEIND,51769.93,-27.18,43.0,X-MC,6.73,82260.0,-1250.0,21281.0,0.45,-1.50,25.87,23.98,82.0,-0.06,0.57,4.24,X40,ATH,APPARELS
10,BAJAJHFL,181.50,-14.06,48.0,X-MC,5.93,180941.0,-19565.0,116725.0,-0.35,-9.76,64.51,48.45,90.0,-0.17,1.26,4.34,X40N,ATH,FINANCE
20,COLPAL,3767.14,-3.45,49.0,X-MC,7.48,225260.0,-38105.0,151465.0,-0.12,-14.47,67.24,43.04,84.0,-0.25,1.57,4.58,XY25,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-47.11,44.0,X-SC,37.82,52768.0,-12238.0,163164.0,-2.07,-18.83,309.21,232.18,198.0,-0.08,0.37,2.93,XY24,NTT,MISC
51,PAGEIND,51769.93,-27.18,43.0,X-MC,6.73,82260.0,-1250.0,21281.0,0.45,-1.50,25.87,23.98,82.0,-0.06,0.57,4.24,X40,ATH,APPARELS
57,RELAXO,1176.00,-39.88,44.0,X-SC,4.67,82006.0,-63154.0,139082.0,-1.14,-43.51,169.60,52.31,136.0,-0.45,0.57,9.77,X40N,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-11.46,32.0,X-MC,2.37,87465.0,-3213.0,18123.0,-1.50,-3.54,20.72,16.44,101.0,-0.18,0.61,15.92,X40,ATH,PHARMA
12,BATAINDIA,2096.00,-40.58,28.0,X-SC,5.36,88785.0,-39885.0,85180.0,-0.21,-31.00,95.94,35.20,219.0,-0.47,0.62,1.91,X40,NTT,FOOTWEAR


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4389.97,-27.72,55.0,X-LC,10.58,287527.0,-58429.0,125132.0,0.77,-16.89,43.52,19.28,1.0,-0.47,2.01,6.29,X40,ATH,IT
39,INFY,2275.00,-18.52,53.0,X-LC,5.50,316795.0,3907.0,167775.0,-0.44,1.25,52.96,54.87,3.0,0.02,2.21,9.63,X40,BTT,IT
41,ITC,452.00,-36.23,64.0,X-LC,1.22,205918.0,5780.0,15114.0,0.56,2.89,7.34,10.44,4.0,0.38,1.44,8.61,X40,NTT,FMCG
83,VBL,671.64,-16.58,54.0,X-LC,8.01,300498.0,-15344.0,128012.0,-2.94,-4.86,42.60,35.67,5.0,-0.12,2.10,8.32,X40N,ATH,FMCG
1,ABB,7934.00,-39.67,53.0,H-LC,5.43,251928.0,-9691.0,128912.0,-0.58,-3.70,51.17,45.57,7.0,-0.08,1.76,7.20,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,137.00,6994.44,57.0,L-SC,8.24,77825.0,-15985.0,92604.0,0.02,-17.04,118.99,81.67,269.0,-0.17,0.54,50.71,XR,NTT,CERAMICS
19,COFFEEDAY,80.00,-45.67,49.0,L-SC,28.34,81370.0,-32179.0,72232.0,0.64,-28.34,88.77,35.27,268.0,-0.45,0.57,99.15,XR,NTT,HOTELS
49,MASFIN,398.61,-19.15,51.0,H-SC,8.74,91965.0,-6015.0,27617.0,-0.60,-6.14,30.03,22.05,152.0,-0.22,0.64,33.68,XR,ATH,FINANCE
85,WHIRLPOOL,2270.00,-38.01,62.0,M-SC,4.03,128450.0,13652.0,80384.0,-1.74,11.89,62.58,81.92,223.0,0.17,0.90,52.06,XR,NTT,DURABLES
36,INDIAMART,4810.62,-53.33,55.0,H-SC,6.52,130401.0,7065.0,124559.0,-0.37,5.73,95.52,106.72,119.0,0.06,0.91,30.45,AR,ATH,MISC


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
46,KPIGREEN,731.05,12.48,74.0,H-SC,5.26,134203.0,8906.0,50017.0,-0.35,7.11,37.27,47.03,141.0,0.18,0.94,68.03,MH,ATH,POWER
81,VAIBHAVGBL,521.00,76.31,73.0,H-SC,2.20,152748.0,-30027.0,141093.0,-1.16,-16.43,92.37,60.77,125.0,-0.21,1.07,36.81,XR,NTT,JEWELLERY
5,ANGELONE,3033.00,16.33,64.0,X-SC,5.68,206312.0,15306.0,42397.0,0.12,8.01,20.55,30.21,157.0,0.36,1.44,30.57,X40N,NTT,FINANCE
85,WHIRLPOOL,2270.00,-38.01,62.0,M-SC,4.03,128450.0,13652.0,80384.0,-1.74,11.89,62.58,81.92,223.0,0.17,0.90,52.06,XR,NTT,DURABLES
62,SAMMAANCAP,326.00,-196.59,74.0,M-SC,30.14,166905.0,17685.0,126497.0,0.58,11.85,75.79,96.62,208.0,0.14,1.17,80.63,XY25,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.78
1,25,44.42
2,50,75.58


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.23
LC    32.58
MC    23.16
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.16
X40      16.29
XY25     12.10
X40N     11.78
XR        9.98
AR        7.85
OX40N     6.57
X200      1.79
X5K       1.45
SR        1.06
MH        0.94
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.42
X-LC    21.78
X-MC    18.81
M-SC    12.77
X-SC     5.59
M-LC     5.15
H-LC     4.64
H-MC     2.38
M-MC     1.65
L-SC     1.45
L-LC     1.01
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.51,-3.77,37.63
IT,12.53,-17.14,79.96
FINANCE,12.16,-8.64,57.89
MISC,7.93,-14.40,72.11
ELECTRICAL,5.82,-8.55,47.03
PAINTS,5.53,-16.68,34.08
BANKS,4.10,-15.07,82.00
INSURANCE,3.86,0.02,37.26
AUTO,2.78,-44.60,104.95


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3258052.0,22
XR,1227754.0,13
AR,1165516.0,9
X40,823147.0,12
X40N,641403.0,9
OX40N,542614.0,9
XY25,450829.0,8
SR,268936.0,2
X5K,109882.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3333603.0,24
M-SC,1217080.0,15
X-MC,1184634.0,14
X-LC,1021420.0,13
X-SC,548367.0,6
H-LC,274457.0,3
M-LC,272454.0,4
L-SC,253925.0,3
H-MC,246721.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1171815.0      6
           AR         833893.0      5
           XR         767695.0      7
M-SC       XY24       695462.0      6
X-MC       XY24       576279.0      4
X-LC       X40        450302.0      5
           XY24       303816.0      2
X-SC       X40N       300023.0      4
X-MC       X40        287665.0      6
H-SC       SR         268936.0      2
           OX40N      241247.0      3
H-LC       AR         206830.0      2
X-LC       X40N       191046.0      3
H-MC       XY24       176987.0      1
X-MC       XY25       170356.0      2
L-SC       XR         164836.0      2
X-SC       XY24       163164.0      1
M-MC       XY24       156610.0      1
X-MC       X40N       150334.0      2
M-SC       OX40N      142544.0      4
           XR         127784.0      2
           XY25       126497.0      1
           AR         124793.0      2
M-LC       XR         110515.0      1
           X5K        109882.0      1
L-SC       OX40N       89089.0      1
X-SC       X40         85180.0      1
X-LC       XY25        76256.0      3
H-MC       OX40N       69734.0      1
H-LC       X200        67627.0      1
L-MC       XR          56924.0      1
H-SC       MH          50017.0      1
L-LC       XY25        39582.0      1
M-LC       XY25        38138.0      1
           XY24        13919.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 30.0 seconds
